# Aircontrol Primitive Communication

This is just for to debug things. Not to be used as actual API

## Importing Requirements

In [1]:
from pprint import pprint
import PIL.Image as Image
import base64
import numpy as np
from io import BytesIO
import socket
import json
from airctrl import sample_generator

sample = sample_generator.samples()

In [2]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(("127.0.0.1", 8053))


In [3]:

def send_data( data_dict: dict):
        """
        Send all data to the server
        
        Args:
            data_dict (dict): Dict with data
            sock (socket): Socket connection aquired from `get_socket` method
        """
        data = json.dumps(data_dict)
        sock.sendall(data.encode("utf-8"))

    

In [4]:
def receive_data():
        """
        Receives data from server
        
        Args:
            sock (socket): Socket connection aquired from `get_socket` method
        Returns:
            data(dict): Data Received from the server
        """
        BUFF_SIZE = 4096  # 4 KiB
        data = b""
        while True:
            part = sock.recv(BUFF_SIZE)
            data += part
            if len(part) < BUFF_SIZE:
                # either 0 or end of data
                break
        data = eval(data)
        return data

In [5]:
def bool2string(booltype):
        """
        Convert python booll to string
        Such string can be consumed by C# and parsed as bool agin

        Args:
            booltype (bool): 'True' or 'False'

        Returns:
            True: "true"
            False:"false" 
        """
        if booltype:
            return "true"
        else:
            return "false"


In [6]:
control_schema = {
            "MsgType": "ControlInput",
            "InputControlType": "Code",
            "Pitch": 0.0,
            "Roll": 0.0,
            "Yaw": 0.0,
            "Throttle": 0.0,
            "StickyThrottle": 0.5,
            "Brake": 0,
            "Flaps": 0,
            "IsOutput": True,
        }


In [7]:
reset_schema = {
    "MsgType": "Level",
    "InputControlType": "Code",
    "IsActive": bool2string(True),
    "LevelReload":bool2string(True),
    "IsOutput": bool2string(False),
        }
audio_schema = {
            "MsgType": "Audio",
            "IsActive": bool2string(True),
            "EnableAudio": bool2string(False),
            "IsOutput": bool2string(False),
        }
ui_schema = {
            "MsgType": "UI",
            "IsActive": bool2string(True),
            "ShowUIElements": bool2string(True),
            "IsOutput": bool2string(False),
        }
camera_schema = {
            "MsgType": "Camera",
            "IsActive": bool2string(True),
            "ActiveCamera": 1,
            "IsCapture": bool2string(False),
            "CaptureCamera": 1,
            "CaptureType": 1,
            "CaptureWidth": 250,
            "CaptureHeight": 250,
            "IsOutput": bool2string(True),
        }

def step(Pitch, Yaw, Roll, StickyThrottle):
    Pitch = np.clip(Pitch, -1, 1)
    Roll = np.clip(Roll, -1, 1)
    Yaw = np.clip(Yaw, -1, 1)
    Throttle = np.clip(0.65, 0, 1)
    StickyThrottle = np.clip(StickyThrottle, 0, 1)
    Brake = np.clip(0.0, 0, 1)
    Flaps = max(min(0.0, 2), 0)

    control_schema = {
                "MsgType": "ControlInput",
                "InputControlType": "Code",
                "Pitch": Pitch,
                "Roll": Roll,
                "Yaw": Yaw,
                "Throttle": Throttle,
                "StickyThrottle": StickyThrottle,
                "Brake": Brake,
                "Flaps": Flaps,
                "IsOutput": bool2string(True),
    }
    send_data(control_schema) 
    output =  receive_data()
    return output


In [8]:
def reset():
    # reset
    send_data(reset_schema) 
    output =  receive_data()
    # audio
    send_data(audio_schema) 
    output =  receive_data()
    # Ui 
    send_data(ui_schema) 
    output =  receive_data()
    # camera 
    send_data(camera_schema) 
    output =  receive_data()
    return output

In [9]:
def output2features(output):
    """
    output2features(output)
    Args:
        ```
        output ([type]): [description]
        >>> ([0.905434847,
        0.00182869844,
        0.000590562,
        0.0,
        0.0,
        0.021583642933333334,
        0.325318575,
        0.0460257],
        True)
        ```
    Returns:
        [type]: flight status
        [bool]: if collided
    """
    MSL = output['MSL']
    Latitude = output['Latitude']
    Longitude = output['Longitude']
    normalizedRPM =  output['CurrentRPM']/output['MaxRPM']
    normalizedPower =  output['CurrentPower']/output['MaxPower']
    normalizedSpeed = output['CurrentSpeed']/150
    pitchAngle = output['PitchAngle']
    bankAngle = output['BankAngle']
    ifCollision = output['IfCollision']
    collidedWith = output['CollisionObject']
    Reward = output["Reward"]
    feature_vector = [MSL, Latitude, Longitude, normalizedRPM, normalizedPower, normalizedSpeed, pitchAngle, bankAngle]
    return np.asarray(feature_vector),Reward, ifCollision,collidedWith
    

In [10]:
# watch an untrained agent
output = reset()
features,reward, ifCollided,_ = output2features(output)
eps = 1.0
while (True):
        action = [sample.get_random_pitch(), sample.get_random_yaw(), sample.get_random_roll(), sample.get_random_stickythrottle()]
        pitch =  action[0]
        yaw = action[1]
        roll= action[2]
        stickyThrottle=action[3]
        # print(pitch, yaw, roll, stickyThrottle)
        output = step(Pitch=pitch, Yaw=yaw, Roll=roll, StickyThrottle=stickyThrottle)
        features,reward, ifCollided,collidedWith = output2features(output)
        del output["LidarPointCloud"]
        print( output["Counter"], " ifcollided : ", ifCollided," --- ", collidedWith, output)
        if ifCollided != "false":
            print("-> Resetted : ", output["Counter"], " | ifcollided : ", ifCollided," --- ", collidedWith, output )
            output = reset()


control_schema {'MsgType': 'ControlInput', 'InputControlType': 'Code', 'Pitch': -0.017170419405889037, 'Roll': 0.9940563489341521, 'Yaw': 0.22232515400784103, 'Throttle': 0.65, 'StickyThrottle': 0.931432107957055, 'Brake': 0.0, 'Flaps': 0.0, 'IsOutput': 'true'}
2  ifcollided :  false  ---   {'AGL': 0.0, 'MSL': 39.7567673, 'CurrentRPM': 0.0, 'MaxRPM': 3000.0, 'MaxPower': 3500.0, 'CurrentPower': 0.0, 'CurrentFuel': 0.0, 'CurrentSpeed': 0.0, 'BankAngle': 0.00422646524, 'IfCollision': 'false', 'collisionObject': '', 'Latitude': -2.130081e-06, 'Longitude': 5.40877045e-06, 'PitchAngle': 0.3172346, 'ScreenCapture': '', 'Counter': 2, 'log': '', 'Reward': 0.0, 'MsgType': 'Output', 'Version': '0.0.5'}
control_schema {'MsgType': 'ControlInput', 'InputControlType': 'Code', 'Pitch': 0.49511335568602255, 'Roll': 0.9563424890991237, 'Yaw': -0.5603902006684223, 'Throttle': 0.65, 'StickyThrottle': 0.8217589081297746, 'Brake': 0.0, 'Flaps': 0.0, 'IsOutput': 'true'}
3  ifcollided :  false  ---   {'AGL': 

KeyboardInterrupt: 

In [23]:
receive_data()

timeout: timed out

In [ ]:
output